In [176]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.model_selection import LeaveOneOut

## List of all the stations and variables available in DBHydro portal

In [126]:
Station_list = ["POLESOUT", "KISSR0.0", "LZ2", "S133", "TCNS228", "S135",
                "FEBIN", "MBOXSOU", "MH24000", "FEBOUT", "L005", "L008", "LZ40", "L004", "S308C",
                "PALMOUNT", "S169", "S236", "POLE3S", "RITTAE2", "LZ2FA", "L007", "PELBAY3", "L006", "LZ30"]
Variable_list = ['NITRATE+NITRITE-N', 'NITRITE-N', 'AMMONIA-N', 'KJELDAHL NITROGEN, TOTAL', 'PHOSPHATE, ORTHO AS P',
                 'PHOSPHATE, TOTAL AS P', 'NITRATE-N', 'SILICA', 'CARBON, TOTAL ORGANIC', 'CARBON, DISSOLVED ORGANIC',
                 'TOTAL NITROGEN', 'PHOSPHATE, DISSOLVED AS P', 'KJELDAHL NITROGEN, DIS', 'CARBON, TOTAL', 
                 'CARBON, TOTAL INORGANIC', 'NITROGEN, TOTAL DISSOLVED', 'CHLOROPHYLL-A(LC)']

Others = ['CHLOROPHYLL-A', 'PHEOPHYTIN', 'CHLOROPHYLL-A, CORRECTED', 
         'CHLOROPHYLL-C', 'CAROTENOIDS', 'CHLOROPHYLL-B', 'CHLOROPHYLL-A(LC)', 
         'PHEOPHYTIN-A(LC)', 'CHLOROPHYLL-B(LC)', 'RESP. PLANKTONIC']

Nitrogen_list = ['NITRATE+NITRITE-N', 'NITRITE-N', 'AMMONIA-N', 'KJELDAHL NITROGEN, TOTAL',
                'TOTAL NITROGEN','NITRATE-N','KJELDAHL NITROGEN, DIS','NITROGEN, TOTAL DISSOLVED']
Phosphorus_list = ['PHOSPHATE, ORTHO AS P','PHOSPHATE, TOTAL AS P','PHOSPHATE, DISSOLVED AS P']
Carbon_list = ['CARBON, TOTAL ORGANIC', 'CARBON, DISSOLVED ORGANIC','CARBON, TOTAL','CARBON, TOTAL INORGANIC']

## The dictionary of predictors should use for each station:

In [130]:
summary = pd.read_csv("Variables_summary.csv")
Predictor_dict = dict({
    "POLESOUT":[],
    "KISSR0.0":[],
    "LZ2":[],
    "L005":[],
    "L008":[],
    "LZ40":[],
    "S308C":[],
    "POLE3S":[],
    "RITTAE2":[],
    "L007":[],
    "PELBAY3":[],
    "L006":[],
    "LZ30":[]
})

for station in list(Predictor_dict.keys()):
    station_summary = summary.loc[summary["Station"]==station, :]
    for variable in station_summary["Variable"]:
        if list(station_summary["Measurements in 2019"][station_summary["Variable"]==variable])[0] >= 10 and variable != 'CHLOROPHYLL-A(LC)' and variable not in Others:
            Predictor_dict[station].append(variable)

In [5]:
def Clean_date(df):
    """
    Transfer value in "Collection_Date" column in to Datetime object
    """
    for index, value in enumerate(df["Collection_Date"]):
        df["Collection_Date"][index] = datetime.strptime(value, '%Y-%m-%d %H:%M:%S')        
    return df

Double check the unit!

In [117]:
def Data_clean_up(data):
    """
    The columns of the final table:
    - Time (month-year)
    - Value of variable 1
    .
    .
    - Value of variable n
    """
    #clean date: 
    data = Clean_date(data)
    
    Time_array = []
    for year in range(2000, 2020):
        for month in range(1,13):
            Time_array.append(datetime(year, month, 1))
    
    clean_data = pd.DataFrame({"MonthYear": Time_array})
    
    global Variable_list
    for variable in Variable_list:
        Value_array = []
        for time in Time_array:
            #select rows based on the given date and variable:
            index_given_time = [True if (x.month == time.month and x.year == time.year) else False for x in data["Collection_Date"]]
            index_given_variable = data["Test Name"] == variable
            #combine 2 criteria:
            index_to_choose = index_given_time & index_given_variable
            
            values = data.loc[index_to_choose, "Value"]
            #Filter negative and NA values:
            values = [x for x in values if (not math.isnan(x)) and x >= 0]
         
            if len(values) > 0:
                Value_array.append(np.mean(values))
                
            else:
                Value_array.append(None)
                
        clean_data[variable] = Value_array
        
    return clean_data

In [21]:
station = "POLESOUT"
data = Data_clean_up(pd.read_csv(station+".csv"))
data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,MonthYear,NITRATE+NITRITE-N,NITRITE-N,AMMONIA-N,"KJELDAHL NITROGEN, TOTAL","PHOSPHATE, ORTHO AS P","PHOSPHATE, TOTAL AS P",NITRATE-N,SILICA,"CARBON, TOTAL ORGANIC",...,CHLOROPHYLL-A,PHEOPHYTIN,"CHLOROPHYLL-A, CORRECTED",CHLOROPHYLL-C,CAROTENOIDS,CHLOROPHYLL-B,CHLOROPHYLL-A(LC),PHEOPHYTIN-A(LC),CHLOROPHYLL-B(LC),RESP. PLANKTONIC
0,2000-01-01,0.324,NaN,0.010,0.999,0.046,0.116,NaN,NaN,None,...,20.2,3.7,17.5,4.4,10.9,2.3,NaN,NaN,NaN,None
1,2000-02-01,0.437,NaN,NaN,1.570,0.044,0.167,NaN,10.486,None,...,7.2,NaN,6.6,1.0,5.0,NaN,NaN,NaN,NaN,None
2,2000-03-01,0.301,NaN,NaN,1.218,0.046,0.130,NaN,NaN,None,...,32.2,NaN,30.8,1.9,12.5,2.1,NaN,NaN,NaN,None
3,2000-04-01,0.007,NaN,NaN,2.512,0.008,0.184,NaN,NaN,None,...,6.2,NaN,5.5,NaN,3.0,NaN,NaN,NaN,NaN,None
4,2000-05-01,0.351,0.004,NaN,1.398,0.020,0.116,0.347,11.242,None,...,28.3,8.8,22.0,NaN,12.6,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019-08-01,NaN,NaN,0.014,NaN,NaN,0.048,NaN,4.060,None,...,NaN,NaN,NaN,NaN,NaN,NaN,74.7,0.565,NaN,None
236,2019-09-01,NaN,0.002,0.028,NaN,0.002,0.101,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,59.6,1.450,NaN,None
237,2019-10-01,NaN,0.002,0.019,NaN,0.010,0.152,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,28.9,2.640,NaN,None
238,2019-11-01,0.012,NaN,0.014,NaN,0.002,0.046,0.012,4.220,None,...,NaN,NaN,NaN,NaN,NaN,NaN,19.5,0.587,NaN,None


## Linear Regression for POLESOUT station

In [8]:
from sklearn.linear_model import LinearRegression

#Indentify indenpdent variables (predictors):
predictors = ["NITRATE+NITRITE-N", "NITRITE-N", "AMMONIA-N", "PHOSPHATE, TOTAL AS P"]
regression_table = pd.DataFrame({"Variable":predictors})

for station in ["POLESOUT", "KISSR0.0", "S308C", "LZ30"]:
    data = pd.read_csv(station+".csv")
    data = Data_clean_up(data)
    index_to_choose = [True for x in range(len(data))]
    #Identify the datapoints have NAs value to exclude (False value in index_to_choose)
    for variable in predictors + ["CHLOROPHYLL-A(LC)"]:
        for index, value in enumerate(data[variable]):
            if pd.isna(value):
                index_to_choose[index] = False

    X = data.loc[index_to_choose, predictors]
    y = data.loc[index_to_choose, "CHLOROPHYLL-A(LC)"]
    if len(X) > 20:
        regression = LinearRegression().fit(X,y)
        coefresult = regression.coef_

        regression_table[station] = coefresult

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,18,26,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doi

In [9]:
regression_table

,Variable,POLESOUT,KISSR0.0,LZ30
0,NITRATE+NITRITE-N,-74.723657,-48.454017,-101.069967
1,NITRITE-N,-1365.592606,-34.038308,-21.353697
2,AMMONIA-N,-448.197635,-87.642632,-142.723360
3,"PHOSPHATE, TOTAL AS P",96.938254,58.982903,275.544136


In [22]:
station = "KISSR0.0"
data = Data_clean_up(pd.read_csv(station+".csv"))
data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,MonthYear,NITRATE+NITRITE-N,NITRITE-N,AMMONIA-N,"KJELDAHL NITROGEN, TOTAL","PHOSPHATE, ORTHO AS P","PHOSPHATE, TOTAL AS P",NITRATE-N,SILICA,"CARBON, TOTAL ORGANIC",...,CHLOROPHYLL-A,PHEOPHYTIN,"CHLOROPHYLL-A, CORRECTED",CHLOROPHYLL-C,CAROTENOIDS,CHLOROPHYLL-B,CHLOROPHYLL-A(LC),PHEOPHYTIN-A(LC),CHLOROPHYLL-B(LC),RESP. PLANKTONIC
0,2000-01-01,0.071250,NaN,0.035500,0.881667,0.022250,0.076333,NaN,NaN,NaN,...,11.35,2.850,9.250,NaN,4.450,NaN,NaN,NaN,NaN,None
1,2000-02-01,0.149500,NaN,0.020000,1.060000,0.036000,0.115000,NaN,7.122000,NaN,...,15.70,NaN,15.200,NaN,6.600,NaN,NaN,NaN,NaN,None
2,2000-03-01,0.343000,NaN,NaN,1.456333,0.059667,0.141000,NaN,NaN,NaN,...,11.85,3.550,9.450,2.800000,8.350,1.3,NaN,NaN,NaN,None
3,2000-04-01,0.168500,NaN,NaN,1.387667,0.033000,0.157000,NaN,NaN,NaN,...,8.60,1.300,7.600,NaN,3.900,NaN,NaN,NaN,NaN,None
4,2000-05-01,0.212714,NaN,NaN,1.342667,0.021833,0.101250,NaN,6.109000,NaN,...,31.00,6.875,25.725,2.233333,11.675,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019-08-01,0.005500,0.003333,0.026750,NaN,0.002000,0.090667,NaN,4.603333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,32.733333,1.733333,NaN,None
236,2019-09-01,0.027000,0.007000,0.115000,NaN,0.048000,0.094000,0.02100,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.000000,0.805000,NaN,None
237,2019-10-01,0.016000,0.021000,0.026000,NaN,0.021000,0.089000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16.200000,1.420000,NaN,None
238,2019-11-01,0.128250,0.005333,0.015333,NaN,0.018000,0.078667,0.12425,4.836667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,27.466667,1.250000,NaN,None


In [133]:
def count_num_points(data, station):
    global Predictor_dict
    num_points = pd.DataFrame()

    for number_of_remove in range(1, len(Predictor_dict[station])+1):
        combs = combinations(Predictor_dict[station], number_of_remove)
        for list_to_remove in list(combs):
            new_row = dict({i:True for i in Predictor_dict[station]})

            for remove_var in list_to_remove:
                new_row[remove_var] = False

            num_points = num_points.append(new_row, ignore_index=True)

    num_points['CHLOROPHYLL-A(LC)'] = [True for i in range(len(num_points))]
    num_data = []
    for i in range(len(num_points)):
        list_variable = []
        for variable in num_points.columns:
            if num_points[variable][i]:
                list_variable.append(variable)

        data2 = data.loc[:,list_variable]

        index_to_choose = [True for x in range(len(data2))]
        #Identify the datapoints have NAs value to exclude (False value in index_to_choose)
        for variable in data2.columns:
            for index, value in enumerate(data2[variable]):
                if pd.isna(value):
                    index_to_choose[index] = False
        num_data.append(np.sum(index_to_choose))

    num_points["Num data"] = num_data
    return num_points

In [144]:
station = "POLESOUT"
data = Data_clean_up(pd.read_csv(f"{station}.csv"))
count_table = count_num_points(data, station).sort_values("Num data", ascending=False).reset_index()
count_table

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,index,AMMONIA-N,NITRATE+NITRITE-N,NITRATE-N,NITRITE-N,"PHOSPHATE, ORTHO AS P","PHOSPHATE, TOTAL AS P",TOTAL NITROGEN,CHLOROPHYLL-A(LC),Num data
0,126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,109
1,121,0.0,0.0,0.0,0.0,0.0,1.0,0.0,True,109
2,123,1.0,0.0,0.0,0.0,0.0,0.0,0.0,True,104
3,106,1.0,0.0,0.0,0.0,0.0,1.0,0.0,True,104
4,122,0.0,0.0,0.0,0.0,1.0,0.0,0.0,True,75
...,...,...,...,...,...,...,...,...,...,...
122,10,1.0,0.0,1.0,1.0,1.0,0.0,1.0,True,13
123,9,1.0,0.0,1.0,1.0,0.0,1.0,1.0,True,13
124,4,1.0,1.0,1.0,1.0,1.0,0.0,1.0,True,13
125,3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,True,13


In [179]:
best_predictors = []
i = 40
best_row = count_table.loc[[i],:]
for predictor in Predictor_dict[station]:
    if best_row[predictor][i]:
        best_predictors.append(predictor)
        
index_to_choose = [True for x in range(len(data))]
#Identify the datapoints have NAs value to exclude (False value in index_to_choose)
for variable in best_predictors + ["CHLOROPHYLL-A(LC)"]:
    for index, value in enumerate(data[variable]):
        if pd.isna(value):
            index_to_choose[index] = False
            
X = data.loc[index_to_choose, best_predictors].reset_index(drop=True)
y = data.loc[index_to_choose,"CHLOROPHYLL-A(LC)"].reset_index(drop=True)

regression_table = pd.DataFrame({"Predictor":best_predictors})

def FullLinearRegression(X,y):
    """
    Return the coefficients and cross validation score (leave one out)
    """
    regression = LinearRegression().fit(X,y)
    coeffs = regression.coef_

    cv_scores = []

    for train_index, test_index in LeaveOneOut().split(X):
        X_train, X_test = X.iloc[np.array(train_index),:], X.iloc[np.array(test_index),:]
        y_train, y_test = y[train_index], y[test_index]
        regression = LinearRegression().fit(X_train, y_train)
        y_pred = regression.predict(X_test)
        cv_scores.append(mean_squared_error(y_test, y_pred))
        
    return coeffs, cv_scores